In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from utils.utils import load_data, remove_zero_features, load_confounders, standardize, label_freq_sorted, pca_transform
from utils.utils import generate_oversampled_set, generate_undersampled_set, generate_label_stats
from utils.utils import compute_scores

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain

from sklearn.model_selection import train_test_split

In [ ]:
N_CORES = joblib.cpu_count(only_physical_cores=True)
print(f"Number of physical cores: {N_CORES}")

Number of physical cores: 8


---

In [ ]:
# Load data for classification task
subject_data, features, diagnoses = load_data('classification')

In [ ]:
# Remove zero features
F = remove_zero_features(features.iloc[:,1:])

In [ ]:
# Load confounders
C = load_confounders(subject_data)

In [ ]:
# Standardize
X = standardize(F)
print(f"Number of samples: {X.shape[0]}")
print(f"Number of features: {X.shape[1]}")

Number of samples: 2815
Number of features: 922


In [ ]:
# Remove ID column
Y = diagnoses.iloc[:,1:]
print(f"Number of labels: {Y.shape[1]}")

Number of labels: 13


In [ ]:
boot_iter = 100

# 3. Use oversampled dataset

In [ ]:
# Resample data (undersampling)
X_over, Y_over = generate_oversampled_set(X, Y)
label_stats, mean_ir = generate_label_stats(Y_over, True)
print(f"Mean imbalance ratio: {mean_ir}")
label_stats

Mean imbalance ratio: 1.6092872677464145


,Absolute frequency,Relative frequency,Imbalance ratio
Trauma_And_Stress_RelatedDisorders,2580,0.116174,1.923256
DepressiveDisorders,3170,0.142741,1.565300
Attention_Deficit_HyperactivityDisorder,4582,0.206322,1.082933
MotorDisorder,3134,0.141120,1.583280
AutismSpectrumDisorder,3689,0.166111,1.345080
CommunicationDisorder,4431,0.199523,1.119838
OtherDisorders,2320,0.104467,2.138793
SpecificLearningDisorder,4962,0.223433,1.000000
Obsessive_Compulsive_And_RelatedDisorders,2668,0.120137,1.859820
Disruptive,2801,0.126126,1.771510


In [ ]:
# Split dataset into train and test (holdout) set
X_train, X_test, Y_train, Y_test = train_test_split(X_over, Y_over, test_size=0.25, random_state=0)

# Confounder 
C_train, C_test = C.loc[X_train.index], C.loc[X_test.index]

# PCA features
X_pca = pca_transform(F)
X_pca_train, X_pca_test = X_pca.loc[X_train.index], X_pca.loc[X_test.index]

print(f"Number of samples in training set: {len(X_train)}")
print(f"Number of samples in test set: {len(X_test)}")

Number of samples in training set: 16656
Number of samples in test set: 5552


## 3.1. MultiOutputClassifier
Evaluate classification models wrapped in meta estimator MultiOutputClassifier with respect to multi-label performance metrics

### 3.1.1. Dummy estimators

#### 3.1.1.1. Always zero baseline estimator

In [ ]:
clf = DummyClassifier(strategy='constant', constant=0 ,random_state=0)
meta_clf = MultiOutputClassifier(clf).fit(X_train, Y_train)
compute_scores(meta_clf, X_test, Y_test, boot_iter)

Mean scores with SE and 95% confidence intervals:

auprc_macro:                  0.15 (0.00) [0.15, 0.15]
auprc_weighted:               0.16 (0.00) [0.16, 0.17]
auroc_macro:                  0.50 (0.00) [0.50, 0.50]
auroc_weighted:               0.50 (0.00) [0.50, 0.50]
brier_macro:                  0.15 (0.00) [0.15, 0.15]
brier_weighted:               0.02 (0.00) [0.02, 0.02]
balanced_accuracy_macro:      0.50 (0.00) [0.50, 0.50]
balanced_accuracy_weighted:   0.07 (0.00) [0.07, 0.07]
f1_micro:                     0.00 (0.00) [0.00, 0.00]
hamming:                      0.15 (0.00) [0.15, 0.15]
subset_accuracy:              0.01 (0.00) [0.01, 0.02]


#### 3.1.1.2. Label proportion baseline estimator

In [ ]:
clf = DummyClassifier(strategy='prior', random_state=0)
meta_clf = MultiOutputClassifier(clf).fit(X_train, Y_train)
compute_scores(meta_clf, X_test, Y_test, boot_iter)

Mean scores with SE and 95% confidence intervals:

auprc_macro:                  0.15 (0.00) [0.15, 0.15]
auprc_weighted:               0.16 (0.00) [0.16, 0.17]
auroc_macro:                  0.50 (0.00) [0.50, 0.50]
auroc_weighted:               0.50 (0.00) [0.50, 0.50]
brier_macro:                  0.13 (0.00) [0.12, 0.13]
brier_weighted:               0.02 (0.00) [0.02, 0.02]
balanced_accuracy_macro:      0.50 (0.00) [0.50, 0.50]
balanced_accuracy_weighted:   0.07 (0.00) [0.07, 0.07]
f1_micro:                     0.00 (0.00) [0.00, 0.00]
hamming:                      0.15 (0.00) [0.15, 0.15]
subset_accuracy:              0.01 (0.00) [0.01, 0.02]


### 3.1.2. Confounders as features

#### 3.1.1.1. Logistic regression

In [ ]:
clf = LogisticRegression(max_iter=10000, random_state=0)
meta_clf = MultiOutputClassifier(clf).fit(C_train, Y_train)
compute_scores(meta_clf, C_test, Y_test, boot_iter)

Mean scores with SE and 95% confidence intervals:

auprc_macro:                  0.20 (0.00) [0.19, 0.20]
auprc_weighted:               0.21 (0.00) [0.20, 0.21]
auroc_macro:                  0.58 (0.00) [0.58, 0.59]
auroc_weighted:               0.57 (0.00) [0.57, 0.58]
brier_macro:                  0.12 (0.00) [0.12, 0.13]
brier_weighted:               0.02 (0.00) [0.02, 0.02]
balanced_accuracy_macro:      0.50 (0.00) [0.50, 0.50]
balanced_accuracy_weighted:   0.07 (0.00) [0.07, 0.07]
f1_micro:                     0.00 (0.00) [0.00, 0.00]
hamming:                      0.15 (0.00) [0.15, 0.15]
subset_accuracy:              0.01 (0.00) [0.01, 0.02]


#### 3.1.1.1. Histogram-based Gradient Boosting

In [ ]:
clf = HistGradientBoostingClassifier(random_state=0)
meta_clf = MultiOutputClassifier(clf, n_jobs=-1).fit(C_train,Y_train)
compute_scores(meta_clf, C_test, Y_test, boot_iter)

Mean scores with SE and 95% confidence intervals:

auprc_macro:                  0.84 (0.00) [0.84, 0.85]
auprc_weighted:               0.83 (0.00) [0.82, 0.84]
auroc_macro:                  0.95 (0.00) [0.94, 0.95]
auroc_weighted:               0.94 (0.00) [0.94, 0.94]
brier_macro:                  0.05 (0.00) [0.05, 0.05]
brier_weighted:               0.01 (0.00) [0.01, 0.01]
balanced_accuracy_macro:      0.84 (0.00) [0.83, 0.84]
balanced_accuracy_weighted:   0.12 (0.00) [0.12, 0.12]
f1_micro:                     0.75 (0.01) [0.74, 0.76]
hamming:                      0.06 (0.00) [0.06, 0.07]
subset_accuracy:              0.65 (0.01) [0.64, 0.66]


### 3.1.3. PCA-projected data (top-10 components) as features 

#### 3.1.1.1. Logistic regression

In [ ]:
clf = LogisticRegression(max_iter=10000, random_state=0)
meta_clf = MultiOutputClassifier(clf).fit(X_pca_train, Y_train)
compute_scores(meta_clf, X_pca_test, Y_test, boot_iter)

Mean scores with SE and 95% confidence intervals:

auprc_macro:                  0.28 (0.01) [0.27, 0.29]
auprc_weighted:               0.28 (0.01) [0.27, 0.29]
auroc_macro:                  0.65 (0.00) [0.65, 0.66]
auroc_weighted:               0.64 (0.00) [0.63, 0.65]
brier_macro:                  0.12 (0.00) [0.12, 0.12]
brier_weighted:               0.02 (0.00) [0.02, 0.02]
balanced_accuracy_macro:      0.51 (0.00) [0.51, 0.51]
balanced_accuracy_weighted:   0.08 (0.00) [0.08, 0.08]
f1_micro:                     0.04 (0.00) [0.04, 0.05]
hamming:                      0.15 (0.00) [0.15, 0.15]
subset_accuracy:              0.02 (0.00) [0.02, 0.02]


#### 3.1.1.1. Histogram-based Gradient Boosting

In [ ]:
clf = HistGradientBoostingClassifier(random_state=0)
meta_clf = MultiOutputClassifier(clf).fit(X_pca_train,Y_train) # n_jobs=-1
compute_scores(meta_clf, X_pca_test, Y_test, boot_iter)

Mean scores with SE and 95% confidence intervals:

auprc_macro:                  0.97 (0.00) [0.97, 0.97]
auprc_weighted:               0.97 (0.00) [0.96, 0.97]
auroc_macro:                  0.99 (0.00) [0.99, 0.99]
auroc_weighted:               0.99 (0.00) [0.99, 0.99]
brier_macro:                  0.02 (0.00) [0.02, 0.02]
brier_weighted:               0.00 (0.00) [0.00, 0.00]
balanced_accuracy_macro:      0.94 (0.00) [0.93, 0.94]
balanced_accuracy_weighted:   0.14 (0.00) [0.14, 0.14]
f1_micro:                     0.92 (0.00) [0.91, 0.92]
hamming:                      0.02 (0.00) [0.02, 0.02]
subset_accuracy:              0.83 (0.00) [0.82, 0.84]


### 3.1.4. Original features (standardized)

#### 3.1.4.1. Logistic regression

In [ ]:
clf = LogisticRegression(max_iter=10000, random_state=0)
meta_clf = MultiOutputClassifier(clf).fit(X_train, Y_train) # n_jobs=-1
compute_scores(meta_clf, X_test, Y_test, boot_iter)

Mean scores with SE and 95% confidence intervals:

auprc_macro:                  0.88 (0.00) [0.87, 0.89]
auprc_weighted:               0.87 (0.00) [0.86, 0.88]
auroc_macro:                  0.96 (0.00) [0.96, 0.96]
auroc_weighted:               0.96 (0.00) [0.95, 0.96]
brier_macro:                  0.03 (0.00) [0.03, 0.03]
brier_weighted:               0.01 (0.00) [0.01, 0.01]
balanced_accuracy_macro:      0.92 (0.00) [0.91, 0.92]
balanced_accuracy_weighted:   0.14 (0.00) [0.14, 0.14]
f1_micro:                     0.86 (0.00) [0.86, 0.87]
hamming:                      0.04 (0.00) [0.04, 0.04]
subset_accuracy:              0.76 (0.01) [0.75, 0.77]


#### 3.1.4.2. SVM

In [ ]:
clf = SVC(class_weight='balanced', kernel='rbf', gamma='scale', probability=True, random_state=0)
meta_clf = MultiOutputClassifier(clf, n_jobs=-1).fit(X_train, Y_train)
compute_scores(meta_clf, X_test, Y_test, 10)

#### 3.1.4.3. Histogram-based Gradient Boosting

In [ ]:
clf = HistGradientBoostingClassifier(random_state=0)
meta_clf = MultiOutputClassifier(clf).fit(X_train, Y_train) # n_jobs=-1
compute_scores(meta_clf, X_test, Y_test, boot_iter)

Mean scores with SE and 95% confidence intervals:

auprc_macro:                  0.23 (0.01) [0.20, 0.25]
auprc_weighted:               0.33 (0.01) [0.30, 0.35]
auroc_macro:                  0.61 (0.01) [0.58, 0.63]
auroc_weighted:               0.59 (0.01) [0.57, 0.61]
brier_macro:                  0.13 (0.00) [0.12, 0.13]
brier_weighted:               0.03 (0.00) [0.03, 0.03]
balanced_accuracy_macro:      0.51 (0.00) [0.50, 0.52]
balanced_accuracy_weighted:   0.08 (0.00) [0.08, 0.08]
f1_micro:                     0.17 (0.01) [0.15, 0.20]
hamming:                      0.15 (0.00) [0.14, 0.16]
subset_accuracy:              0.12 (0.01) [0.09, 0.14]


#### 3.1.4.4. MLP

In [ ]:
clf = MLPClassifier(random_state=0)
meta_clf = MultiOutputClassifier(clf).fit(X_train, Y_train) # n_jobs=-1
compute_scores(meta_clf, X_test, Y_test, boot_iter)

Mean scores with SE and 95% confidence intervals:

auprc_macro:                  0.20 (0.01) [0.18, 0.22]
auprc_weighted:               0.30 (0.01) [0.28, 0.33]
auroc_macro:                  0.57 (0.01) [0.55, 0.60]
auroc_weighted:               0.56 (0.01) [0.53, 0.58]
brier_macro:                  0.16 (0.00) [0.15, 0.16]
brier_weighted:               0.04 (0.00) [0.03, 0.04]
balanced_accuracy_macro:      0.52 (0.01) [0.51, 0.53]
balanced_accuracy_weighted:   0.08 (0.00) [0.08, 0.08]
f1_micro:                     0.25 (0.01) [0.23, 0.28]
hamming:                      0.18 (0.00) [0.17, 0.19]
subset_accuracy:              0.06 (0.01) [0.04, 0.08]


## 3.2. ClassifierChain (ordered by frequency) 
Evaluate classification models wrapped in meta estimator ClassifierChain with respect to multi-label performance metrics

In [ ]:
by_freq = label_freq_sorted(Y_train)

### 3.2.1. Logistic regression

In [ ]:
clf = LogisticRegression(max_iter=10000, random_state=0)
meta_clf = ClassifierChain(clf, order=by_freq, random_state=0).fit(X_train, Y_train)
compute_scores(meta_clf, X_test, Y_test, boot_iter, chain=True)

Mean scores with SE and 95% confidence intervals:

auprc_macro:                  0.20 (0.01) [0.18, 0.21]
auprc_weighted:               0.30 (0.01) [0.27, 0.32]
auroc_macro:                  0.57 (0.01) [0.54, 0.59]
auroc_weighted:               0.55 (0.01) [0.53, 0.58]
brier_macro:                  0.18 (0.00) [0.17, 0.19]
brier_weighted:               0.04 (0.00) [0.04, 0.04]
balanced_accuracy_macro:      0.52 (0.01) [0.51, 0.54]
balanced_accuracy_weighted:   0.08 (0.00) [0.08, 0.08]
f1_micro:                     0.29 (0.01) [0.27, 0.32]
hamming:                      0.21 (0.01) [0.20, 0.22]
subset_accuracy:              0.06 (0.01) [0.04, 0.07]


### 3.2.2. Histogram-based Gradient Boosting

In [ ]:
clf = HistGradientBoostingClassifier(random_state=0)
meta_clf = ClassifierChain(clf, order=by_freq, random_state=0).fit(X_train, Y_train)
compute_scores(meta_clf, X_test, Y_test, boot_iter, chain=True)

## 3.3. ClassifierChain (random order) 
Evaluate classification models wrapped in meta estimator ClassifierChain with respect to multi-label performance metrics

### 3.3.1. Logistic regression

In [ ]:
clf = LogisticRegression(max_iter=10000, random_state=0)
meta_clf = ClassifierChain(clf, random_state=0).fit(X_train, Y_train)
compute_scores(meta_clf, X_test, Y_test, boot_iter, chain=True)

Mean scores with SE and 95% confidence intervals:

auprc_macro:                  0.21 (0.01) [0.20, 0.22]
auprc_weighted:               0.36 (0.01) [0.34, 0.38]
auroc_macro:                  0.57 (0.01) [0.55, 0.59]
auroc_weighted:               0.56 (0.01) [0.54, 0.58]
brier_macro:                  0.21 (0.00) [0.21, 0.22]
brier_weighted:               0.05 (0.00) [0.04, 0.05]
balanced_accuracy_macro:      0.53 (0.01) [0.51, 0.54]
balanced_accuracy_weighted:   0.09 (0.00) [0.09, 0.09]
f1_micro:                     0.34 (0.01) [0.32, 0.36]
hamming:                      0.26 (0.00) [0.26, 0.28]
subset_accuracy:              0.02 (0.01) [0.01, 0.03]


### 3.3.2. Histogram-based Gradient Boosting

In [ ]:
clf = HistGradientBoostingClassifier(random_state=0)
meta_clf = ClassifierChain(clf, random_state=0).fit(X_train, Y_train)
compute_scores(meta_clf, X_test, Y_test, boot_iter, chain=True)